In [1]:
import sys
print(sys.version)

3.10.5 (main, Jun 15 2022, 12:41:18) [Clang 13.1.6 (clang-1316.0.21.2.5)]


In [48]:
import os
from pathlib import Path
from sys import stderr

In [49]:
cd ~/builds/dotfiles/

/Users/wolf/builds/dotfiles


In [50]:
def mark_all_children_not_already_marked(source_package_root: Path, exclude_dirs: list[str], parent: Path, mark: str, plan: dict[Path, str]):
    leaf = source_package_root / parent
    for child in leaf.iterdir():
        f = child.relative_to(source_package_root)
        if should_exclude_this_object(child, exclude_dirs):
            continue
        if f not in plan:
            plan[f] = mark

def mark_all_parents(leaf: Path, mark: str, stop_mark: str, plan: dict[Path, str]):
    for parent in leaf.parents:
        if parent in plan and plan[parent] == stop_mark:
            break
        plan[parent] = mark

def should_exclude_this_object(directory: Path, exclude_dirs: list[str])->bool:
    dir_list = directory.parts
    for exclude_dir in exclude_dirs:
        if exclude_dir in dir_list:
            return True
    return False

In [51]:
def debug_print_plan(source_package_root: Path, plan: dict[Path, str]):
    for dirpath, dirnames, filenames in os.walk(source_package_root):
        full_target = Path(dirpath)
        target = full_target.relative_to(source_package_root)
        
        if target in plan:
            print(f"{target}: '{plan[target]}'")
            for filename in filenames:
                file = target / filename
                if file in plan:
                    print(f"{file}: '{plan[file]}'")

In [57]:
def plan_install(source_package_root: Path, destination: Path, exclude_dirs: list[str]) -> dict[Path, str]:
    plan: dict[Path, str] = {}

    for dirpath, dirnames, filenames in os.walk(source_package_root, False):
        full_target = Path(dirpath)
        if should_exclude_this_object(full_target, exclude_dirs):
            continue

        target = full_target.relative_to(source_package_root)
        dest = destination_root / target

        if full_target == source_package_root:
            plan[target] = "exists"
        elif list(full_target.glob("dot-*")):
            plan[target] = "create"
            mark_all_parents(target, mark="create", stop_mark="exists", plan=plan)
        elif dest.exists:
            plan[target] = "exists"
            mark_all_parents(target, mark="exists", stop_mark="exists", plan=plan)
        else:
            plan[target] = "link"

        if plan[target] in {"create", "exists"}:
            mark_all_children_not_already_marked(source_package_root, exclude_dirs, target, mark="link", plan=plan)
        elif plan[target] == "link":
            mark_all_children_not_already_marked(source_package_root, exclude_dirs, target, mark="skip", plan=plan)
    
    del plan[Path(".")]
    return plan


In [58]:
exclude_dirs = ['.mypy_cache']
source_package_root = Path('karabiner')
destination_root = Path('~')

plan = plan_install(source_package_root, destination_root, exclude_dirs)

debug_print_plan(source_package_root, plan)

.config: 'exists'
.config/karabiner.edn: 'link'
